In [72]:
import pandas as pd
import csv
from pprint import pprint
import glob
import os
import sys

In [109]:
def transformEAV(raw, index, values="val", columns="group_stat", aggfunc="last", output=None):
    relational_data = raw.pivot_table(
        index=index,
        columns=columns,
        values=values,
        aggfunc=aggfunc
    ).reset_index()
    if output:
        relational_data.to_csv(f'{output}',index=False)
    return relational_data
        

In [111]:
team_season_stats = []

folder_path = 'team_stats/'
csv_files = glob.glob(os.path.join(folder_path,'*_season.csv'))
for file in csv_files:
    filename = os.path.basename(file)
    team_stats = pd.read_csv(file)
    date = filename[:4]
    team_stats['group_stat'] = team_stats['group'] + '_'+team_stats['stat']
    relational_stats=transformEAV(team_stats,index=['season','teamId','domain','range','updateDate'],output=f'team_stats/relational/{date}_team_season_stats.csv')

In [99]:
raw = pd.read_csv('team_stats/2025-07-04_team_stats.csv')
raw["group_stat"] = raw["group"] + "_" + raw["stat"]

In [82]:
team_game_stats = transformEAV(raw,    index=["gamePk", "domain", "range", "side"]).reset_index()

column_names = (team_game_stats.columns.tolist())
team_game_stats.to_csv('team_stats/relational/2025-07-04_team_game_stats.csv',index=False)


In [ ]:
list_season_stats = []
#Combine player season stats
folder_path = 'player_stats/'
csv_files = glob.glob(os.path.join(folder_path,'*_player_season_stats.csv'))
for file in csv_files:
    season_stats = pd.read_csv(file)
    list_season_stats.append(season_stats)

In [95]:
player_game_stats = []
folder_path = 'player_stats/'
#Add player game stats, one row per player per game
csv_files = glob.glob(os.path.join(folder_path, '*_player_game_stats.csv'))
for file in csv_files:
    filename = os.path.basename(file)
    game_stats = pd.read_csv(file)
    date = filename[:10]
    game_stats['group_stat'] = game_stats['stat']+'_'+game_stats['group']
    relational_stats = transformEAV(game_stats, index = ["gamePk","side","playerId"], output=f"player_stats/{date}_game_stats.csv")